<h1 align="center">Advanced Image Processing and Analysis</h1>
<h3 align="center">ECE 4438B/ECE 9022/ECE 9202B/BIOMED 9519B/BIOPHYS 9519B/CAMI 9519B</h3>
<h4 align="center"><a href="mailto:echen29@uwo.ca"> Elvis Chen, PhD, LL</a></h4>
<h4 align="center">Robarts Research Institute, London</h4>
<h4 align="center">Department of Electrical and Computer Engineering, Western University</h4>
<h4 align="center">School of Biomedical Engineering, Western University</h4>
<h4 align="center">Department of Medical Biophysics, Western University</h4>
<h4 align="center">Day 05, January 21, 2019</h4>

#### Introduction
So far, we have covered the following:
* Python as a programming language:
  * numerical operations
  * string operations
  * list operations
  * indexing and slicing of string/list
  * writing a simple module
  * scope of variables
* How to read and write images in SimpleITK
* How to manipulate image as array
  * Direct image construction
  * subsampling using slicing operation
  * cropping/flipping, etc.
* How to display image
  * using external programs such as
    * ImageJ
    * (optional) [ITK-SNAP](http://www.itksnap.org/pmwiki/pmwiki.php)
    * (optional) [3D Slicer](https://www.slicer.org/)

We also have used a few SimpleITK classes (without explanation). In this Notebook, we will examine the architecture of SimpleITK.

In [ ]:
# by now you should know how to load SimpleITK as a module in python
import SimpleITK as sitk

%matplotlib inline
import matplotlib.pyplot as plt

%run update_path_to_download_script
from downloaddata import fetch_data as fdata


from ipywidgets import interact, fixed
import os

OUTPUT_DIR = 'Output'

print(sitk.Version()) # make sure we have loaded SimpleITK successfully and ran a script that allows us to download some data

Here is an example of how to read a image/volume using the SimpleITK interface:

[sitk.ReadImage](https://itk.org/SimpleITKDoxygen100/html/namespaceitk_1_1simple.html#ae3b678b5b043c5a8c93aa616d5ee574c)

Note that we don't really have to worry about file extension, as the class ReadImage automatically figure it out for us.

In [ ]:
# 3D volume example
vol = sitk.ReadImage('..\data\\volumes\grayscale.nrrd')

This Code example reads an image (a [`.nrrd`](https://en.wikipedia.org/wiki/Nrrd) file). This single-line of code actually has two objectives for you to learn:
* in ITK/SimpleITK, every class is a **filter**
  * a filter has an **input**
  * a filter generates an **output**
* filters are connected in a *pipeline* of execution. Pictorially, the pipeline of execution looks like this:

<img src="readerPipeline.png" style="width:500px"/><br><br>

We can convert the file type by reading the file (of a particular format) and write it to a different file (of another file format) using the [WriteImage](https://itk.org/SimpleITKDoxygen100/html/namespaceitk_1_1simple.html#a26db5ae04c6df98d7b072f03fd435fdd) class:

In [ ]:
sitk.WriteImage(vol, os.path.join(OUTPUT_DIR, 'grayscale.nii'))

Pictorially, the pipeline of execution looks like this:

<img src="readWritePipeline.png" style="width:600px"/><br><br>

#### Simple Thresholding as a image processing filter in a pipeline architecture

In terms of computer programming, a [filter](https://en.wikipedia.org/wiki/Filter_\(software%29) is a subroutine (in the case of Python/SimpleITK, a class) that
* receives a [stream](https://en.wikipedia.org/wiki/Stream_(computing%29)
* processes the stream
* produces another stream

While a single filter can be used individually, filters are often connected together to form a [pipeline](https://en.wikipedia.org/wiki/Pipeline_(software%29).

Thus, writing a SimpleITK program is a matter of connecting filters to form a pipeline. The beginning of the pipeline is often a file reader (unless we are constructing the image manually). As SimpleITK has no direct visualization capability, the end of the pipeline is oftean a filter writer.

##### Beginning of the pipeline
Let us perform a simple thresholding using SimpleITK filter. From January 16's lecture, we saw how to threshold an image using direct matrix operation.

Load a CT volume containing bones. As the appearance of bones are bright (i.e. high intensity), segmentation of bone should be relatively easy.

In [ ]:
data_directory = os.path.dirname(fdata("CIRS057A_MR_CT_DICOM/readme.txt"))
# Global variable 'selected_series' is updated by the interact function
selected_series = ''
def DICOM_series_dropdown_callback(series_to_load, series_dictionary):
    global selected_series
               # Print some information about the series from the meta-data dictionary
               # DICOM standard part 6, Data Dictionary: http://medical.nema.org/medical/dicom/current/output/pdf/part06.pdf
    img = sitk.ReadImage(series_dictionary[series_to_load][0])
    tags_to_print = {'0010|0010': 'Patient name: ', 
                     '0008|0060' : 'Modality: ',
                     '0008|0021' : 'Series date: ',
                     '0008|0080' : 'Institution name: ',
                     '0008|1050' : 'Performing physician\'s name: '}
    for tag in tags_to_print:
        try:
            print(tags_to_print[tag] + img.GetMetaData(tag))
        except: # Ignore if the tag isn't in the dictionary
            pass
    selected_series = series_to_load                    

# Directory contains multiple DICOM studies/series, store
             # in dictionary with key being the series ID
reader = sitk.ImageSeriesReader()
series_file_names = {}
series_IDs = reader.GetGDCMSeriesIDs(data_directory)
            # Check that we have at least one series
if series_IDs:
    for series in series_IDs:
        series_file_names[series] = reader.GetGDCMSeriesFileNames(data_directory, series)
    
    interact(DICOM_series_dropdown_callback, series_to_load=list(series_IDs), series_dictionary=fixed(series_file_names)); 
else:
    print('Data directory does not contain any DICOM series.')

We have see this function earlier.  While we typically think an image (such as a .jpeg or .png) or a volume (such as .mhd) is contained in a single file, some [DICOM](https://en.wikipedia.org/wiki/DICOM) files are contained in a series of files, each of which needs to be read indivisually. Each file series is typically stored under the same directory (folder).

The python script above allows us to select the name (ID) of a series, and read all DICOME series under that directory (ID). It would return a 3D volume instead (of a series of 2D images).

The drop-down menu has 3 options: 1 is CT, the other 2 are MRI volume. **Select the CT volume**, which contains the CT of a rib cage phantom.

In [ ]:
reader.SetFileNames(series_file_names[selected_series])
vol = reader.Execute()
# Display the image slice from the middle of the stack, z axis
z = int(vol.GetDepth()/2)
plt.imshow(sitk.GetArrayViewFromImage(vol)[z,:,:], cmap=plt.cm.Greys_r)
plt.axis('off');

what it looks like in 3D:

Transverse view:
<img src="rib_cage_transverse.png" style="width:500px"/><br><br>
Oblique view:
<img src="rib_cage_obliquepng.png" style="width:500px"/><br><br>

Let us take a look at the histogram of this slice.

In [ ]:
# plt.hist(sitk.GetArrayViewFromImage(vol)[z,:,:])
plt.figure()
plt.hist(sitk.GetArrayViewFromImage(vol)[z,:,:])
plt.title("Histogram")
plt.show()

You can try to generate the histogram for the entire 3D volume. It will probably take a long time

**Question**: What is the command (slicing operation) to generate the histogram for the entire volume?

[**Hounsfield scale**](https://en.wikipedia.org/wiki/Hounsfield_scale), or **CT number**, is a quantitative scale for describing radiodensity. When we read a CT volume, the number associated with each voxel is stored in the Hounsfield scale: The Hounsfield scale of distilled water at standared pressure and temperature (STP) is defined as `zero` Hounsfield unit (HU), while the radiodensity of air at STP is defined at -1000 HU.

In [ ]:
img = vol[:,:,z]
# plt.show(img)

In [ ]:
sitk.Show(img)
help(sitk.Threshold)

One notable difference using ImageJ to display the image, instead of using plt, is that the image is displayed at its native (pixel) size.

In [ ]:
segImg = sitk.Threshold(img, 200, 600, 0)
sitk.Show(segImg)

now save the result into a file. The pipeline will look like this:
    
<img src="readThresholdWritePipeline.png" style="width:800px"/><br><br>

In [ ]:
# the complete image thredholding program
reader.SetFileNames(series_file_names[selected_series])
vol = reader.Execute()
segVol = sitk.Threshold(vol, 200, 600, 0)
sitk.WriteImage(vol, os.path.join(OUTPUT_DIR, 'segmentedCT.nrrd'))

Just for interest, let us take a look at the histogram of the segmented image

In [ ]:
plt.figure()
# plt.hist(sitk.GetArrayViewFromImage(segVol)[z,:,:])
plt.hist( segImg )
plt.title("Histogram")
plt.xlabel("Pixel Intensity")
plt.ylabel("Occurance")
plt.show()

In [ ]:
help( plt.xlabel)

# Gaussian filter (Image smoothing example)

Let us write a complete program using the following pipeline:
<img src="gaussianPipeline.png" style="width:800px"/><br><br>

**Let us be a bit more interactive**: 
* Take a picture and upload it to your computer.
* Move the file to the Jupyter Notebook directory
* **What is the command to load this file into python/SimpleITK?**

In [ ]:
# INSERT CODE HERE

# load the image, and save it to a variable named `img`

In [ ]:
pixelID = img.GetPixelIDValue() # why are we doing this?

In [ ]:
gaussian = sitk.SmoothingRecursiveGaussianImageFilter()

In [ ]:
help(sitk.SmoothingRecursiveGaussianImageFilter)

In [ ]:
help(sitk.SmoothingRecursiveGaussianImageFilter.SetSigma)

In [ ]:
gaussian.SetSigma( 5 )
outImg = gaussian.Execute( img )
def myshow(img):
    nda = sitk.GetArrayViewFromImage(img)
    plt.imshow(nda, cmap=plt.cm.Greys_r)
myshow(outImg)

Adjust the Sigma value, and observe how it effect the smoothing appearance.

Once you are satisfied, save the image to a file.

In [ ]:
sitk.WriteImage( sitk.Cast( outImg, pixelID ), 'smoothImage.png')